# Data Processing Template

Since almost without exception the first step is working with data munging, this notebook is stubbed out with a very select few imports to speed up the initial steps following creating a project and copying in raw data.

In [1]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry
import pandas as pd
import os
import importlib

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# a couple of handy variables and settings to get started

data = r'../data'
data_raw = os.path.join(data, 'raw')
data_ext = os.path.join(data, 'external')
data_int = os.path.join(data, 'interim')
data_out = os.path.join(data, 'processed')

gdb_int = os.path.join(data_int, 'interim.gdb')
gdb_raw = os.path.join(data_raw, 'raw.gdb')

In [3]:
bp_pth = os.path.join(gdb_raw, 'block_points')
bg_pth = os.path.join(gdb_raw, 'block_groups')
cbsa_pth = os.path.join(gdb_raw, 'cbsas')

arcpy.env.overwriteOutput = True

In [10]:
bp_id_df = GeoAccessor.from_featureclass(bp_pth)
bp_id_df.head()

,OBJECTID,FIPS,STATE_NAME,SHAPE
0,1,150039812001004,Hawaii,"{""x"": -166.28822369699998, ""y"": 23.86846842700..."
1,2,150039812001005,Hawaii,"{""x"": -166.21047832799997, ""y"": 23.78768193900..."
2,3,150039812001006,Hawaii,"{""x"": -166.24250323499996, ""y"": 23.87162798700..."
3,4,150039812001007,Hawaii,"{""x"": -166.24162861199997, ""y"": 23.87150556400..."
4,5,150039812001008,Hawaii,"{""x"": -166.28719940299996, ""y"": 23.86874607300..."


In [15]:
bp_id_df['bg_id'] = bp_id_df.FIPS.apply(lambda val: val[:12])

In [17]:
bp_id_df['x'] = bp_id_df.SHAPE.apply(lambda geom: geom.x)
bp_id_df['y'] = bp_id_df.SHAPE.apply(lambda geom: geom.y)
bp_id_df.head()

,OBJECTID,FIPS,STATE_NAME,SHAPE,bg_id,x,y
0,1,150039812001004,Hawaii,"{""x"": -166.28822369699998, ""y"": 23.86846842700...",150039812001,-166.288224,23.868468
1,2,150039812001005,Hawaii,"{""x"": -166.21047832799997, ""y"": 23.78768193900...",150039812001,-166.210478,23.787682
2,3,150039812001006,Hawaii,"{""x"": -166.24250323499996, ""y"": 23.87162798700...",150039812001,-166.242503,23.871628
3,4,150039812001007,Hawaii,"{""x"": -166.24162861199997, ""y"": 23.87150556400...",150039812001,-166.241629,23.871506
4,5,150039812001008,Hawaii,"{""x"": -166.28719940299996, ""y"": 23.86874607300...",150039812001,-166.287199,23.868746


In [25]:
bg_cent_df = bp_id_df[['bg_id', 'x', 'y']].groupby('bg_id').mean()
bg_cent_df['SHAPE'] = bg_cent_df.apply(lambda row: Geometry({'x': row[0], 'y': row[1], 'spatialReference': {'wkid': 4326}}), axis=1)
bg_cent_df.head()

,x,y,SHAPE
bg_id,,,
010010201001,-86.488317,32.466165,"{'x': -86.48831683333329, 'y': 32.466164500000..."
010010201002,-86.489838,32.482278,"{'x': -86.48983799999996, 'y': 32.482278047826..."
010010202001,-86.473257,32.477616,"{'x': -86.47325653999998, 'y': 32.477616286666..."
010010202002,-86.471881,32.463169,"{'x': -86.47188078857143, 'y': 32.463168645714..."
010010203001,-86.458227,32.477925,"{'x': -86.45822720952376, 'y': 32.477924514285..."


In [26]:
bg_cent_df.spatial.set_geometry('SHAPE')

In [27]:
bg_cent_df.reset_index(inplace=True)
bg_cent_df.head()

,bg_id,x,y,SHAPE
0,010010201001,-86.488317,32.466165,"{""x"": -86.48831683333329, ""y"": 32.466164500000..."
1,010010201002,-86.489838,32.482278,"{""x"": -86.48983799999996, ""y"": 32.482278047826..."
2,010010202001,-86.473257,32.477616,"{""x"": -86.47325653999998, ""y"": 32.477616286666..."
3,010010202002,-86.471881,32.463169,"{""x"": -86.47188078857143, ""y"": 32.463168645714..."
4,010010203001,-86.458227,32.477925,"{""x"": -86.45822720952376, ""y"": 32.477924514285..."


In [28]:
bg_cent_df.spatial.to_featureclass(os.path.join(gdb_int, 'bg_pop_cent'))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\bg_pop_cent'